# 02: Ingeniería de Características para Ayatsil (v1)

* **Autor:** Emmanuel Eduardo
* **Fecha de Versión:** 2025-07-19
* **Versión:** 1.0 (Simple)
* **Proyecto:** Pronóstico de Producción - CNH

## Objetivo
Este notebook toma la serie de tiempo del campo **Ayatsil**, crea un conjunto de **características simples** (rezagos a corto plazo, promedios móviles) y guarda el resultado en la capa GOLD.

## Entradas
* `gold/top_10_campos_timeseries.parquet`

## Salidas
* `gold/ayatsil_features_simple_v1.parquet`

In [27]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [29]:
# =======================
# 0. Definir ruta base del proyecto
# =======================
ruta_base = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
ruta_gold = os.path.join(ruta_base, "GOLD")

In [31]:
df_top_10 = pd.read_parquet(os.path.join(ruta_gold, "top_10_campos_timeseries.parquet"))

In [33]:
# --- 3. Ingeniería de Características para Ayatsil (Versión Simple) ---
campo_elegido = "AYATSIL"
df_campo = df_top_10[df_top_10['campo'] == campo_elegido].copy()
df_campo['fecha'] = pd.to_datetime(df_campo['fecha'])
df_campo.set_index('fecha', inplace=True)

# Creación de las "pistas" o features
df_campo['produccion_mes_anterior'] = df_campo['petroleo_mbd'].shift(1)
df_campo['produccion_promedio_3m'] = df_campo['petroleo_mbd'].rolling(window=3).mean()
df_campo['mes'] = df_campo.index.month
df_campo['trimestre'] = df_campo.index.quarter
print(f"Ingeniería de características simple completada para el campo: {campo_elegido}.")

Ingeniería de características simple completada para el campo: AYATSIL.


In [35]:
# --- 4. Limpieza y Guardado del Dataset Procesado ---
# Eliminar filas con valores nulos generados por las operaciones de ventana (shift y rolling)
df_modelo = df_campo.dropna()

# Definir el nombre del archivo de salida
nombre_archivo_salida = "ayatsil_features_for_training.parquet"
ruta_completa_salida = os.path.join(ruta_gold, nombre_archivo_salida)

In [37]:
# Guardar el DataFrame final en la capa GOLD
df_modelo.to_parquet(ruta_completa_salida)

print(f"\nProceso finalizado.")
print(f"DataFrame con características para {campo_elegido} guardado exitosamente en:")
print(ruta_completa_salida)




Proceso finalizado.
DataFrame con características para AYATSIL guardado exitosamente en:
C:\Users\elezr\Datalquemy\Proyectos\CNH\GOLD\ayatsil_features_for_training.parquet
